# Classifying names with a character-level RNN
In this notebook we will use a recurrent neural network to predict the language from which certain surnames originate. When given some surname the network outputs a probability distribution over 18 possible languages corresponding to the likelyhood that they originate from these languages. 

This exercise was taken from the [PyTorch website](https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html).

### Download the dataset
The dataset that is used can be downloaded [here](https://download.pytorch.org/tutorial/data.zip). Extract it to the directory where this notebook is located.
Included in the ``data/names`` directory are 18 text files named as
"[Language].txt". Each file contains a bunch of names, one name per
line, mostly romanized (but we still need to convert from Unicode to
ASCII).

If you are running this notebook on Colab you can access the dataset by storing it on your Drive.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


**Change the following path variable such that it points to the location of the dataset**

In [3]:
import os

path_to_data = './gdrive/My Drive/DL_1920/codes/3_tutorial/labs_RNN/lab01_rnn/'  # TODO -- set this to the right location!

os.listdir(path_to_data + '/data/names/')

['Arabic.txt',
 'Irish.txt',
 'Chinese.txt',
 'Italian.txt',
 'Dutch.txt',
 'English.txt',
 'Czech.txt',
 'Greek.txt',
 'German.txt',
 'French.txt',
 'Japanese.txt',
 'Russian.txt',
 'Polish.txt',
 'Spanish.txt',
 'Scottish.txt',
 'Portuguese.txt',
 'Vietnamese.txt',
 'Korean.txt']

### Preparing the data
We first preprocess the dataset by limiting ourselves to ASCII characters

In [4]:
import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))



Slusarski


In [0]:
# Build a dictionary containing a list of names for each language
names_per_language = dict()
languages = list()  # Keep a list containing all languages

def readNames(file_path):  # Define a function that reads all names from some file in /data/names/ and converts them to ASCII
  with open(file_path, encoding='utf-8') as f:
    unicode_names = f.read().strip().split('\n')  # Split the file on new lines. Each line contains a name (in unicode)
    return [unicodeToAscii(name) for name in unicode_names]  # Convert all names to ASCII

# For all files in /data/names/ read the names. Group the names by the language they are in

for filename in os.listdir(path_to_data + 'data/names/'):
  language, _ = filename.split('.')  # Remove the file extention to obtain the class label (the language)
  languages.append(language)
  names = readNames(path_to_data + 'data/names/' + filename)  # Read the names in the current file
  names_per_language[language] = names

n_languages = len(languages)

Now we have ``names_per_language``, a dictionary mapping each language to a list of names.

In [0]:
print(names_per_language['Italian'][:5])

['Abandonato', 'Abatangelo', 'Abatantuono', 'Abate', 'Abategiovanni']


### Exercise - Transforming names into suitable inputs

Now that we have all the names organized, we need to turn them into
Tensors to make any use of them.

To represent a single letter, we use a "one-hot vector" of size
``<1 x n_letters>``. A one-hot vector is filled with 0s except for a 1
at index of the current letter, e.g. ``"b" = <0 1 0 0 0 ...>``.

To make a word we join a bunch of those into a 2D matrix
``<line_length x 1 x n_letters>``.

That extra 1 dimension is because PyTorch assumes everything is in
batches - we're just using a batch size of 1 here.


In [8]:
import torch
import torch.nn.functional as F

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('J'))

print(lineToTensor('Jones').size())

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])
torch.Size([5, 1, 57])


### Exercise - Define a RNN architecture

Create a neural network that takes as input some encoding of a character as well as a hidden state tensor. These two tensors are concatenated and passed to the following:
* a linear layer that produces the next hidden state tensor (no activation function)
* a linear layer that produces an output tensor (no activation function)

In [0]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)

        return output, hidden

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_languages)

To run a step of this network we need to pass an input (in our case, the
Tensor for the current letter) and a previous hidden state (which we
initialize as zeros at first). We'll get back the output and a next hidden state (which we keep for the next
step).


In [21]:
input_tensor = lineToTensor('Albert')
hidden = torch.zeros(1, n_hidden)  # Initialize the hidden state as zeros

print(input_tensor.shape)  # The name contains 6 characters which are all encoded as 1-hot vectors of length 57 (corresponding to all possible input characters)
print(input_tensor[0].shape)  # Show the shape of a single character. The 1 is the batch dimension. In this example we set the batch size to 1 for simplicity
print(hidden.shape)  # Show the shape of the hidden state vector

output, next_hidden = rnn(input_tensor[0], hidden)  # Pass the first letter in the name to the network, as well as the initial hidden state

print(F.softmax(output, dim=1))

torch.Size([6, 1, 57])
torch.Size([1, 57])
torch.Size([1, 128])
tensor([[0.0544, 0.0509, 0.0516, 0.0619, 0.0582, 0.0548, 0.0525, 0.0546, 0.0526,
         0.0541, 0.0622, 0.0542, 0.0553, 0.0562, 0.0523, 0.0588, 0.0597, 0.0556]],
       grad_fn=<SoftmaxBackward>)


### Training

First we will define a function to sample random data points from the train set

In [27]:
import random

def random_train_example():
  # Select a random name in some random language
  language = random.choice(languages)
  name = random.choice(names_per_language[language])
  # Convert the name to a suitable input tensor
  name_tensor = lineToTensor(name)
  # Convert the language to a suitable target tensor
  lang_tensor = torch.LongTensor([languages.index(language)])  # The tensor datatype is 'long' as it contains an integer corresponding to the index of the language
  return language, name, lang_tensor, name_tensor

for i in range(10):
    language, name, lang_tensor, name_tensor = random_train_example()

    print(f'Example {i}, Name: {name}, Language: {language}')


Example 0, Name: Chang, Language: Chinese
Example 1, Name: Bagaryakov, Language: Russian
Example 2, Name: Mojzis, Language: Czech
Example 3, Name: Escarra, Language: Spanish
Example 4, Name: Mikhail, Language: Arabic
Example 5, Name: Cousineau, Language: French
Example 6, Name: Denisovsky, Language: Russian
Example 7, Name: Schepansky, Language: Russian
Example 8, Name: Lian, Language: Chinese
Example 9, Name: Antar, Language: Arabic


Next we define a function that performs stochastic gradient descent using a single data point

In [0]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(rnn.parameters(), lr=0.005)

def train_on_example(name_tensor, language_tensor):
  hidden_state = torch.zeros(1, n_hidden)

  optimizer.zero_grad()

  for character_tensor in name_tensor:  # Perform a forward pass for each character in the name
    out, hidden_state = rnn(character_tensor, hidden_state)

  loss = criterion(out, language_tensor)
  loss.backward()

  optimizer.step()

  return out, loss.item()



Now iterate through the dataset to train the network

In [29]:

n_iters = 200000

for i in range(1, n_iters + 1):
  language, name, lang_tensor, name_tensor = random_train_example()
  output, loss = train_on_example(name_tensor, lang_tensor)

  if not i % 1000:
    lang_pred = languages[torch.argmax(output).item()]

    print(f'Example {i}, Loss: {loss:.3f} Name: {name:16s} Language: {language:16s} Classified as: {lang_pred:16s} {"Correct" if lang_pred == language else "Incorrect"}')


Example 1000, Loss: 2.815 Name: Chuhonkin        Language: Russian          Classified as: Chinese          Incorrect
Example 2000, Loss: 2.727 Name: Ramecker         Language: Dutch            Classified as: German           Incorrect
Example 3000, Loss: 2.893 Name: Ho               Language: Korean           Classified as: Portuguese       Incorrect
Example 4000, Loss: 2.773 Name: Ruaidh           Language: Irish            Classified as: German           Incorrect
Example 5000, Loss: 2.573 Name: Peter            Language: Dutch            Classified as: German           Incorrect
Example 6000, Loss: 2.805 Name: Sauvageot        Language: French           Classified as: Greek            Incorrect
Example 7000, Loss: 1.604 Name: Vanshenkin       Language: Russian          Classified as: Russian          Correct
Example 8000, Loss: 1.723 Name: Otsuka           Language: Japanese         Classified as: Japanese         Correct
Example 9000, Loss: 2.805 Name: Merckx           Language: D

### Evaluating the network

We now define a function that gives the network names you can enter manually

In [0]:


def predict(input_name, num_langs=3):
  name_tensor = lineToTensor(input_name)
  hidden_state = torch.zeros(1, n_hidden)
  for character_tensor in name_tensor:
    out, hidden_state = rnn(character_tensor, hidden_state) 

  dist = list(zip(languages, F.softmax(out, dim=1).squeeze()))

  topk_langs = sorted(dist, key=lambda p: p[1].item())[-num_langs:]

  for lang, p in reversed(topk_langs):
    print(f'{lang}, {p.item()}')


In [0]:
while(True):
  predict(input())

Dovesky
Russian, 0.6516695022583008
Czech, 0.17072094976902008
Irish, 0.08755696564912796
Hazaki
Japanese, 0.8061946034431458
Polish, 0.16081468760967255
Arabic, 0.01754830963909626
Brune
Scottish, 0.4690245985984802
French, 0.15954533219337463
German, 0.13154950737953186
Jackson
Scottish, 0.9309751987457275
English, 0.02218683622777462
Greek, 0.016810500994324684
Satoshi
Japanese, 0.5210070013999939
Arabic, 0.17787086963653564
Polish, 0.0789252445101738
Lee
Chinese, 0.6519851684570312
Vietnamese, 0.19339802861213684
Korean, 0.08613181859254837
Hinton
Scottish, 0.6364266872406006
English, 0.18541404604911804
French, 0.041077207773923874
Schmiduber
German, 0.48641714453697205
English, 0.19876696169376373
Russian, 0.08197639882564545
Spek
Czech, 0.3449096381664276
Dutch, 0.2512235939502716
Polish, 0.19834773242473602
